<a href="https://colab.research.google.com/github/tilopez/GO_distance/blob/master/0_GO_sim_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resúmen del trabajo sobre medidas de similaridad semántica

Al trabajar con la ontología genética es necesario una medida que permita comparar términos de la misma, para esto se definen las medidas de similaridad semántica: funciones que dados dos elementos de la ontología (o dos conjuntos de elementos) devuelven un valor que representa su cercanía dentro del grafo. Dentro de las medidas más relevantes se encuentra las de Resnik, Lin y Schlicker (para comparar funciones) y el promedio, el mínimo, el máximo de las anteriores, junto con la medida de coseno (para comparar conjuntos de funciones). Todas estas medidas tienen aciertos y errores con respecto a que información de la ontología logran representar, esto se puede identificar cuando se realizan pruebas, puesto que tampoco se cuenta con una medida estándar de evaluación de dichas medidas. Por esto la importancia de encontrar una medida de similaridad que contenga toda la información de la ontología y que, de ser posible, tenga una forma directa de medir su bondad.


Códigos de para cargar librerías y hacer algunas pruebas.

In [0]:
# instalar goatools y pygraphviz; descargar go
!pip install goatools
!apt-get install graphviz-dev
!pip install pygraphviz
!wget http://purl.obolibrary.org/obo/go/go-basic.obo

     |████████████████████████████████| 15.1MB 151kB/s 
     |████████████████████████████████| 163kB 49.0MB/s 
     |████████████████████████████████| 143kB 47.4MB/s 
     |████████████████████████████████| 215kB 48.7MB/s 
  Created wheel for goatools: filename=goatools-0.9.9-cp36-none-any.whl size=15757842 sha256=7f4cf7197eadb33cd7c71366a1d6add05d5d35d8f4158d862d3e31a63570c072
  Stored in directory: /root/.cache/pip/wheels/f1/a4/fe/f6167cf69ce3b53e45326b10e735cd1007780f2f4522f85a34
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=18ebc06cfbf1c76ba7889947f91d65c10602138353cb542f6d4f022c9cd9dc14
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built goatools wget
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 4.5.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR

In [0]:
# cargar la go
from goatools.obo_parser import GODag
godag=GODag("go-basic.obo")

go-basic.obo: fmt(1.2) rel(2019-11-02) 47,242 GO Terms


In [0]:
# asociaciones entre términos de la GO y anotaciones de los genes
import os
from goatools.associations import dnld_assc
# "tair.gaf" :      arabidopsis (planta)
# "aspgd.gaf" :     aspergillus (hongo)
# "rgd.gaf" :       rata
# "goa_human.gaf" : humano
fin_gaf = os.path.join(os.getcwd(), "tair.gaf")
associations = dnld_assc(fin_gaf, godag)

requests.get(http://current.geneontology.org/annotations/tair.gaf.gz, stream=True)
  WROTE: /content/tair.gaf.gz

  gunzip /content/tair.gaf.gz
HMS:0:00:05.144378 225,323 annotations READ: /content/tair.gaf 
17309 IDs in loaded association branch, BP


In [0]:
# similaridad JC

def JC_sim(g1, g2, go, terms):
  ic_g1 = get_info_content(g1, terms)
  ic_g2 = get_info_content(g2, terms)
  sim_res = resnik_sim(g1, g2, go, terms)
  sim_JC = 1 -( ic_g1 + ic_g2 - 2 * sim_res )
  return(sim_JC)

# similaridad rel

def rel_sim(g1, g2, go, terms):
  sim_lin = lin_sim(g1, g2, go, terms)
  sim_res = resnik_sim(g1, g2, go, terms)
  # obtener p(c_MICA)
  p_mica = 10**(-sim_res)
  sim_rel = sim_lin * (1-p_mica)
  return(sim_rel)

# Análisis de similaridades semánticas

La figura presenta los genes que se van a utilizar en este análisis:
![alt text](https://raw.githubusercontent.com/TiagoNLopez/stuff/master/go_test1_tags.png)



In [0]:
import numpy as np
from goatools.semantic import TermCounts, resnik_sim, lin_sim, get_info_content
termcounts = TermCounts(godag, associations)
display(termcounts)

id_list = ["GO:0048519",
           "GO:0050794",
          
           "GO:0048523",
           "GO:0010941",
          
           "GO:0060548",
           "GO:0043067",
          
           "GO:0043069",
           "GO:0042981"]

sims = np.zeros((len(id_list),len(id_list)))

for fun1 in range(len(id_list)):
  for fun2 in range(fun1):
    sims[fun1,fun2] = lin_sim(id_list[fun1], id_list[fun2], godag, termcounts)
    
np.savetxt("sim_lin.csv",sims,delimiter=",")

In [0]:
# probar distintas anotaciones

#fin_gaf1 = os.path.join(os.getcwd(), "aspgd.gaf")
#fin_gaf2 = os.path.join(os.getcwd(), "rgd.gaf")
#fin_gaf3 = os.path.join(os.getcwd(), "goa_human.gaf")
#fin_gaf = os.path.join(os.getcwd(), "tair.gaf")

associations = dnld_assc(fin_gaf1, godag)
termcounts = TermCounts(godag, associations)
sim_asper1 = lin_sim("GO:0060548", "GO:0043067", godag, termcounts)
sim_asper2 = lin_sim("GO:0043069", "GO:0042981", godag, termcounts)

associations = dnld_assc(fin_gaf2, godag)
termcounts = TermCounts(godag, associations)
sim_rata1 = lin_sim("GO:0060548", "GO:0043067", godag, termcounts)
sim_rata2 = lin_sim("GO:0043069", "GO:0042981", godag, termcounts)

associations = dnld_assc(fin_gaf3, godag)
termcounts = TermCounts(godag, associations)
sim_humano1 = lin_sim("GO:0060548", "GO:0043067", godag, termcounts)
sim_humano2 = lin_sim("GO:0043069", "GO:0042981", godag, termcounts)

associations = dnld_assc(fin_gaf, godag)
termcounts = TermCounts(godag, associations)
sim_arab1 = lin_sim("GO:0060548", "GO:0043067", godag, termcounts)
sim_arab2 = lin_sim("GO:0043069", "GO:0042981", godag, termcounts)

print(" ")
print("GO:0060548 vs GO:0043067")
print("Lin aspergilus : ", sim_asper1)
print("Lin rata       : ", sim_rata1)
print("Lin humano     : ", sim_humano1)
print("Lin arabidopsis: ", sim_arab1)

print(" ")
print("GO:0043069 vs GO:0042981")
print("Lin aspergilus : ", sim_asper2)
print("Lin rata       : ", sim_rata2)
print("Lin humano     : ", sim_humano2)
print("Lin arabidopsis: ", sim_arab2)

NameError: ignored

In [0]:
# graficar
godag.draw_lineage([godag["GO:0043066"], godag["GO:0006915"]], nodecolor="gray14", 
                   edgecolor="cornflowerblue", dpi=300, lineage_img="go_test1.png", 
                   draw_parents=True, draw_children=False)

from IPython.display import Image
Image("go_test1.png")

In [0]:
# graficar
godag.draw_lineage([godag["GO:0060150"]], nodecolor="gray14", 
                   edgecolor="cornflowerblue", dpi=300, lineage_img="go_test2.png", 
                   draw_parents=True, draw_children=False)

from IPython.display import Image
Image("go_test2.png")

In [0]:
id = ['GO:0019222', 'GO:0050794', 'GO:0048518']

print('Gen 1: ', id[0], ' IC: ', get_info_content(id[0], termcounts))
print('Gen 2: ', id[1], ' IC: ', get_info_content(id[1], termcounts))
print('Gen 3: ', id[2], ' IC: ', get_info_content(id[2], termcounts))
print(' ')
print('G1 vs G2: ', lin_sim(id[0], id[1], godag, termcounts))
print('G2 vs G3: ', lin_sim(id[1], id[2], godag, termcounts))
print('G3 vs G1: ', lin_sim(id[2], id[0], godag, termcounts))

In [0]:
from goatools.semantic import TermCounts
termcounts = TermCounts(godag, associations)

go_id = ['GO:0008150',
         
         'GO:0009987',
         'GO:0065007',
        
         'GO:0008219',
         'GO:0050789',
        
         'GO:0012501',
         'GO:0048519',
         'GO:0050794',
        
         'GO:0006915',
         'GO:0048523',
         'GO:0010941',
        
         'GO:0060548',
         'GO:0043067',
        
         'GO:0043069',
         'GO:0042981',
        
         'GO:0043066']

freq = []
for id in go_id:
  freq.append(termcounts.get_term_freq(id))
  
print('     ',freq[0])
print(freq[1],' ',freq[2])
print(freq[3],' ',freq[4])
print(freq[5],' ',freq[6],' ',freq[7])
print(freq[8],' ',freq[9],' ',freq[10])
print('     ',freq[11],' ',freq[12])
print('     ',freq[13],' ',freq[14])
print('     ',freq[15])

      1.0
0.6803181325207871   0.3246397768940763
0.006817125445437174   0.2852863709135981
0.005732582760935806   0.05892681919124103   0.2455198058152146
0.00010328977947632082   0.038165573516500545   0.004286525848267314
      0.0017042813613592936   0.003408562722718587
      0.0013427671331921705   0.0003098693384289625
      0.00025822444869080206


# Similaridad sobre funciones individuales

Los genes propuestos son:

**G1:  GO:0012501**,  **GO:0006915**    

**G2:  GO:0048519**,  **GO:0048523**

**G3:  GO:0050794**,  **GO:0010941**

Estos genes comparten funciones en los mismos niveles, que si bien en el gráfico parecen distintos más adelante se demuestra en código que son los mismos.



---


## Medidas de similaridad entre funciones

Las medidas que voy a utilizar son las siguientes:

**Resnik**
$$ Sim_{Res}(c_1,c_2)=IC(c_{MICA}) $$

**Lin**
$$ Sim_{Lin}(c_1,c_2)=\frac{2\times IC(c_{MICA})}{IC(c_1)+IC(c_2)} $$

**Jiang, Conrath**
$$ Sim_{JC}(c_1,c_2)=1-IC(c_1)+IC(c_2)-2\times IC(c_{MICA}) $$

**Schlicker**
$$ Sim_{Rel}(c_1,c_2)=Sim_{Lin}(c_1,c_2) \times (1-p(c_{MICA})) $$



---


## Mediciones a realizar

Se van a comparar las funciones de **G2** con respecto a **G1** y **G3**, siempre en un mismo nivel. Los resultados se obtienen en el código siguiente.

In [0]:
# comprobar anotaciones y niveles

G1F1_go_id = 'GO:0012501'
G1F2_go_id = 'GO:0006915'

G2F1_go_id = 'GO:0048519'
G2F2_go_id = 'GO:0048523'

G3F1_go_id = 'GO:0050794'
G3F2_go_id = 'GO:0010941'

print('Gen 1')
print(godag[G1F1_go_id])
print(godag[G1F2_go_id])

print('Gen 2')
print(godag[G2F1_go_id])
print(godag[G2F2_go_id])

print('Gen 3')
print(godag[G3F1_go_id])
print(godag[G3F2_go_id])

Gen 1
GO:0012501	level-03	depth-03	programmed cell death [biological_process]
GO:0006915	level-04	depth-04	apoptotic process [biological_process]
Gen 2
GO:0048519	level-03	depth-03	negative regulation of biological process [biological_process]
GO:0048523	level-04	depth-04	negative regulation of cellular process [biological_process]
Gen 3
GO:0050794	level-03	depth-03	regulation of cellular process [biological_process]
GO:0010941	level-04	depth-04	regulation of cell death [biological_process]


In [0]:
# ancestro común más informativo
from goatools.semantic import deepest_common_ancestor
go_root = deepest_common_ancestor([G2F2_go_id, G3F2_go_id], godag)
print(go_root)

GO:0050794


In [0]:
# probar y comparar el contenido de información
from goatools.semantic import TermCounts, get_info_content
termcounts = TermCounts(godag, associations)
infocontent1 = get_info_content(G1F1_go_id, termcounts)
infocontent2 = get_info_content(G1F2_go_id, termcounts)
print('Contenido de información padre ({}) = {}'.format(G1F1_go_id, infocontent1))
print('Contenido de información hijo ({}) = {}'.format(G1F2_go_id, infocontent2))

Contenido de información padre (GO:0012501) = 2.2416496666707166
Contenido de información hijo (GO:0006915) = 3.985942649793393


In [0]:
from goatools.semantic import TermCounts, get_info_content, resnik_sim, lin_sim
termcounts = TermCounts(godag, associations)

# similaridades entre términos de nivel 3
res_G1F1_vs_G2F1 = resnik_sim(G1F1_go_id, G2F1_go_id, godag, termcounts)
res_G2F1_vs_G3F1 = resnik_sim(G3F1_go_id, G2F1_go_id, godag, termcounts)

lin_G1F1_vs_G2F1 = lin_sim(G1F1_go_id, G2F1_go_id, godag, termcounts)
lin_G2F1_vs_G3F1 = lin_sim(G3F1_go_id, G2F1_go_id, godag, termcounts)

jc_G1F1_vs_G2F1 = JC_sim(G1F1_go_id, G2F1_go_id, godag, termcounts)
jc_G2F1_vs_G3F1 = JC_sim(G3F1_go_id, G2F1_go_id, godag, termcounts)

rel_G1F1_vs_G2F1 = rel_sim(G1F1_go_id, G2F1_go_id, godag, termcounts)
rel_G2F1_vs_G3F1 = rel_sim(G3F1_go_id, G2F1_go_id, godag, termcounts)

# similaridades entre términos de nivel 4
res_G1F2_vs_G2F2 = resnik_sim(G1F2_go_id, G2F2_go_id, godag, termcounts)
res_G2F2_vs_G3F2 = resnik_sim(G3F2_go_id, G2F2_go_id, godag, termcounts)

lin_G1F2_vs_G2F2 = lin_sim(G1F2_go_id, G2F2_go_id, godag, termcounts)
lin_G2F2_vs_G3F2 = lin_sim(G3F2_go_id, G2F2_go_id, godag, termcounts)

jc_G1F2_vs_G2F2 = JC_sim(G1F2_go_id, G2F2_go_id, godag, termcounts)
jc_G2F2_vs_G3F2 = JC_sim(G3F2_go_id, G2F2_go_id, godag, termcounts)

rel_G1F2_vs_G2F2 = rel_sim(G1F2_go_id, G2F2_go_id, godag, termcounts)
rel_G2F2_vs_G3F2 = rel_sim(G3F2_go_id, G2F2_go_id, godag, termcounts)

print("Comparaciones contra G2F1: GO:0048519 (lvl 3)")
print("Measure   vs G1F1 GO:0012501  vs G3F1 GO:0050794")
print("--------- ------------------- -------------------")
print("resnik    {sim1:.16f} {sim2:.16f}".format(sim1=res_G1F1_vs_G2F1, sim2=res_G2F1_vs_G3F1))
print("lin       {sim1:.16f} {sim2:.16f}".format(sim1=lin_G1F1_vs_G2F1, sim2=lin_G2F1_vs_G3F1))
print("JC        {sim1:.16f} {sim2:.16f}".format(sim1=jc_G1F1_vs_G2F1, sim2=jc_G2F1_vs_G3F1))
print("relevance {sim1:.16f} {sim2:.16f}".format(sim1=rel_G1F1_vs_G2F1, sim2=rel_G2F1_vs_G3F1))
print(" ")
print("Comparaciones contra G2F2: GO:0048523 (lvl 4)")
print("Measure   vs G1F2 GO:0006915  vs G3F2 GO:0010941")
print("--------- ------------------- -------------------")
print("resnik    {sim1:.16f} {sim2:.16f}".format(sim1=res_G1F2_vs_G2F2, sim2=res_G2F2_vs_G3F2))
print("lin       {sim1:.16f} {sim2:.16f}".format(sim1=lin_G1F2_vs_G2F2, sim2=lin_G2F2_vs_G3F2))
print("JC        {sim1:.16f} {sim2:.16f}".format(sim1=jc_G1F2_vs_G2F2, sim2=jc_G2F2_vs_G3F2))
print("relevance {sim1:.16f} {sim2:.16f}".format(sim1=rel_G1F2_vs_G2F2, sim2=rel_G2F2_vs_G3F2))

Comparaciones contra G2F1: GO:0048519 (lvl 3)
Measure   vs G1F1 GO:0012501  vs G3F1 GO:0050794
--------- ------------------- -------------------
resnik    -0.0000000000000000 0.5447189755507806
lin       -0.0000000000000000 0.5922144342743970
JC        -2.4713366677098763 0.2498374819971887
relevance -0.0000000000000000 0.4232637275176047
 
Comparaciones contra G2F2: GO:0048523 (lvl 4)
Measure   vs G1F2 GO:0006915  vs G3F2 GO:0010941
--------- ------------------- -------------------
resnik    -0.0000000000000000 0.6099134680652128
lin       -0.0000000000000000 0.3221751633240093
JC        -4.4042708568559412 -1.5663958240134228
relevance -0.0000000000000000 0.2430747797862135


## Análisis de resultados

* En muchos casos la similaridad resultó ser 0, esto es porque se seleccionaron genes que tiene como $c_{MICA}$ a la raíz, el cual tiene información igual a cero.

* La similaridad de *Resnik* da el resultado esperado en las comparación entre funciones de un mismo nivel, **aumenta al descender en un nivel del grafo** ya que el ancestro pasa a ser un nodo más específico (en el caso de G2 vs G3).

* En *Lin* también se obtienen valores mayores de las comparaciones entre G2 y G3, **la comparación entre niveles es mayor en el primer caso**.

* *JC* no es una buena medida, en el primer caso da mayor para las funciones más distanciadas. Además, los resultados negativos son confusos.

* En *Relevance* el término de probabiladad del MICA produce una **similaridad mayor en términos con MICAs más específicos y menor en otros casos**.

# Similaridad sobre conjuntos de funciones 

## Medidas de similaridad

En este análisis se van a utilizar las medidas anteriores de Resnik, Lin y Schlicker, comparando entre pares de funciones y calculando el promedio, el máximo y el mínimo, también se utiliza la medida del coseno.

**Coseno:**

$$ CoSim_{A,B} = \frac{A\cdot B}{|A|^2 + |B|^2 - A \cdot B}$$

In [0]:
import numpy as np

# genes
G1_go_id = [G1F1_go_id,G1F2_go_id]
G2_go_id = [G2F1_go_id,G2F2_go_id]
G3_go_id = [G3F1_go_id,G3F2_go_id]

G1vsG2_res = []
G1vsG2_lin = []
G1vsG2_rel = []
G1vsG2_cos = []

G3vsG2_res = []
G3vsG2_lin = []
G3vsG2_rel = []
G3vsG2_cos = []

for func1 in G1_go_id:
  for func2 in G2_go_id:
    G1vsG2_res.append(resnik_sim(func1, func2, godag, termcounts))
    G1vsG2_lin.append(lin_sim(func1, func2, godag, termcounts))
    G1vsG2_rel.append(rel_sim(func1, func2, godag, termcounts))
    
    
for func1 in G3_go_id:
  for func2 in G2_go_id:
    G3vsG2_res.append(resnik_sim(func1, func2, godag, termcounts))
    G3vsG2_lin.append(lin_sim(func1, func2, godag, termcounts))
    G3vsG2_rel.append(rel_sim(func1, func2, godag, termcounts))
    
G1G2_res_avg = sum(G1vsG2_res)/4
G3G2_res_avg = sum(G3vsG2_res)/4
G1G2_lin_avg = sum(G1vsG2_lin)/4
G3G2_lin_avg = sum(G3vsG2_lin)/4
G1G2_rel_avg = sum(G1vsG2_rel)/4
G3G2_rel_avg = sum(G3vsG2_rel)/4

G1G2_res_max = max(G1vsG2_res)
G3G2_res_max = max(G3vsG2_res)
G1G2_lin_max = max(G1vsG2_lin)
G3G2_lin_max = max(G3vsG2_lin)
G1G2_rel_max = max(G1vsG2_rel)
G3G2_rel_max = max(G3vsG2_rel)

G1G2_res_min = min(G1vsG2_res)
G3G2_res_min = min(G3vsG2_res)
G1G2_lin_min = min(G1vsG2_lin)
G3G2_lin_min = min(G3vsG2_lin)
G1G2_rel_min = min(G1vsG2_rel)
G3G2_rel_min = min(G3vsG2_rel)

from goatools.semantic import TermCounts, get_info_content
termcounts = TermCounts(godag, associations)
ic_G1F1 = get_info_content(G1F1_go_id, termcounts)
ic_G1F2 = get_info_content(G1F2_go_id, termcounts)
ic_G2F1 = get_info_content(G2F1_go_id, termcounts)
ic_G2F2 = get_info_content(G2F2_go_id, termcounts)
ic_G3F1 = get_info_content(G3F1_go_id, termcounts)
ic_G3F2 = get_info_content(G3F2_go_id, termcounts)


# se recorre del grafo en amplitud (ver figura),
# este recorrido determina la forma de los vectores
# se pesan los nodos con su contenido de información correspondiente
G1v = [0, 
       0, 0, 0, 
       0, 0, 
       0, 0, 0, 
       ic_G1F1, 0, 0,
       ic_G1F2, 0, 0,
       0, 0,
       0]

G2v = [0, 
       0, 0, 0, 
       0, 0, 
       0, ic_G2F1, 0, 
       0, ic_G2F2, 0,
       0, 0, 0,
       0, 0,
       0]

G3v = [0, 
       0, 0, 0, 
       0, 0, 
       0, 0, ic_G3F1, 
       0, 0, ic_G3F2,
       0, 0, 0,
       0, 0,
       0]

punto1 = np.array(G1v) @ np.array(G2v)
punto2 = np.array(G3v) @ np.array(G2v)

G1_snorm = 0
for x in G1v:
  G1_snorm += x**2
  
G2_snorm = 0
for x in G2v:
  G2_snorm += x**2
  
G3_snorm = 0
for x in G3v:
  G3_snorm += x**2
  
G1G2_cos = punto1 / (G1_snorm + G2_snorm - punto1)
G3G2_cos = punto2 / (G3_snorm + G2_snorm - punto2)

# informar resultados
print("Comparaciones de medidas por grupos, comparando contra G2")
print("   Measure           vs G1               vs G3       ")
print("------------- ------------------- -------------------")
print("resnik    min {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_res_min, sim2=G3G2_res_min))
print("lin       min {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_lin_min, sim2=G3G2_lin_min))
print("relevance min {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_rel_min, sim2=G3G2_rel_min))
print("")
print("resnik    avg {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_res_avg, sim2=G3G2_res_avg))
print("lin       avg {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_lin_avg, sim2=G3G2_lin_avg))
print("relevance avg {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_rel_avg, sim2=G3G2_rel_avg))
print("")
print("resnik    max {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_res_max, sim2=G3G2_res_max))
print("lin       max {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_lin_max, sim2=G3G2_lin_max))
print("relevance max {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_rel_max, sim2=G3G2_rel_max))
print("")
print("cosine        {sim1:.16f}  {sim2:.16f}".format(sim1=G1G2_cos, sim2=G3G2_cos))

Comparaciones de medidas por grupos, comparando contra G2
   Measure           vs G1               vs G3       
------------- ------------------- -------------------
resnik    min -0.0000000000000000  0.5447189755507806
lin       min -0.0000000000000000  0.3028250881078103
relevance min -0.0000000000000000  0.2164332176999425

resnik    avg 0.0000000000000000  0.5773162218079968
lin       avg 0.0000000000000000  0.4546588967294742
relevance avg 0.0000000000000000  0.3341329708341844

resnik    max -0.0000000000000000  0.6099134680652128
lin       max -0.0000000000000000  0.6014209012116800
relevance max -0.0000000000000000  0.4537601583329769

cosine        0.0000000000000000  0.0000000000000000



## Análisis de resultados

* En el caso de los genes que tienen como menor ancestro común al nodo raíz, su similaridad es cero.
* En el caso de la medida de coseno como los genes no tienen términos en común las similaridades se anulan.
* Para las medidas que usan max, min o avg, todos los casos son representativos, ya que los genes que se comparan son cercanos y no contienen términos en común.

# Segundo caso de prueba

En este caso los genes elegidos tienen una función en común:

![alt text](https://raw.githubusercontent.com/TiagoNLopez/stuff/master/go_test2.png)

In [0]:
# comprobar anotaciones y niveles

G1_go_id = ['GO:0008152',
            'GO:0043170',
            'GO:0010467']

G2_go_id = ['GO:0008152',
            'GO:0019222',
            'GO:0060255']

G3_go_id = ['GO:0008152',
            'GO:0009892',
            'GO:0010605']

print('Gen 1')
for fun in G1_go_id:
  print(godag[fun])
print(' ')
  
print('Gen 2')
for fun in G2_go_id:
  print(godag[fun])
print(' ')
  
print('Gen 3')
for fun in G3_go_id:
  print(godag[fun])

Gen 1
GO:0008152	level-01	depth-01	metabolic process [biological_process]
GO:0043170	level-03	depth-03	macromolecule metabolic process [biological_process]
GO:0010467	level-04	depth-04	gene expression [biological_process]
 
Gen 2
GO:0008152	level-01	depth-01	metabolic process [biological_process]
GO:0019222	level-03	depth-03	regulation of metabolic process [biological_process]
GO:0060255	level-04	depth-04	regulation of macromolecule metabolic process [biological_process]
 
Gen 3
GO:0008152	level-01	depth-01	metabolic process [biological_process]
GO:0009892	level-04	depth-04	negative regulation of metabolic process [biological_process]
GO:0010605	level-05	depth-05	negative regulation of macromolecule metabolic process [biological_process]


In [0]:
import numpy as np

# min_sim_res_1(G1 vs G2) min_sim_res_2(G2 vs G3) min_lin_1 min_lin_2 min_rel_1 min_rel_2 cos_1 cos_2
# avg_sim_res_1(G1 vs G2) avg_sim_res_2(G2 vs G3) avg_lin_1 avg_lin_2 avg_rel_1 avg_rel_2 0     0
# max_sim_res_1(G1 vs G2) max_sim_res_2(G2 vs G3) max_lin_1 max_lin_2 max_rel_1 max_rel_2 0     0

sims = np.zeros((3,8))
res = []
lin = []
rel = []

for func1 in G1_go_id:
  for func2 in G2_go_id:
    res.append(resnik_sim(func1, func2, godag, termcounts))
    lin.append(lin_sim(func1, func2, godag, termcounts))
    rel.append(rel_sim(func1, func2, godag, termcounts))
    
sims[0,0]=min(res)
sims[1,0]=sum(res)/9
sims[2,0]=max(res)

sims[0,2]=min(lin)
sims[1,2]=sum(lin)/9
sims[2,2]=max(lin)

sims[0,4]=min(rel)
sims[1,4]=sum(rel)/9
sims[2,4]=max(rel)

res = []
lin = []
rel = []

for func1 in G3_go_id:
  for func2 in G2_go_id:
    res.append(resnik_sim(func1, func2, godag, termcounts))
    lin.append(lin_sim(func1, func2, godag, termcounts))
    rel.append(rel_sim(func1, func2, godag, termcounts))

sims[0,1]=min(res)
sims[1,1]=sum(res)/9
sims[2,1]=max(res)

sims[0,3]=min(lin)
sims[1,3]=sum(lin)/9
sims[2,3]=max(lin)

sims[0,5]=min(rel)
sims[1,5]=sum(rel)/9
sims[2,5]=max(rel)

ic_G1 = []
for func in G1_go_id:
  ic_G1.append(get_info_content(func,termcounts))

ic_G2 = []
for func in G2_go_id:
  ic_G2.append(get_info_content(func,termcounts))

ic_G3 = []
for func in G3_go_id:
  ic_G3.append(get_info_content(func,termcounts))
  
  
# sim coseno
  
G1v = [0, 
       ic_G1[0], 0, 
       0, 0, 0, 
       ic_G1[1], 0, 0, 0, 0, 
       ic_G1[2], 0, 0, 0,
       0, 0,
       0, 0,
       0, 0,
       0,
       0,
       0]

G2v = [0, 
       ic_G2[0], 0, 
       0, 0, 0, 
       0, ic_G2[1], 0, 0, 0, 
       0, ic_G2[2], 0, 0,
       0, 0,
       0, 0,
       0, 0,
       0,
       0,
       0]

G3v = [0, 
       ic_G3[0], 0, 
       0, 0, 0, 
       0, 0, 0, 0, 0, 
       0, 0, ic_G3[1], 0,
       0, ic_G3[2],
       0, 0,
       0, 0,
       0,
       0,
       0]

punto1 = np.array(G1v) @ np.array(G2v)
punto2 = np.array(G3v) @ np.array(G2v)

G1_snorm = 0
for x in G1v:
  G1_snorm += x**2
  
G2_snorm = 0
for x in G2v:
  G2_snorm += x**2
  
G3_snorm = 0
for x in G3v:
  G3_snorm += x**2
  
sims[0,6] = punto1 / (G1_snorm + G2_snorm - punto1)
sims[0,7] = punto2 / (G3_snorm + G2_snorm - punto2)

# min_sim_res_1(G1 vs G2) min_sim_res_2(G2 vs G3) min_lin_1 min_lin_2 min_rel_1 min_rel_2 cos_1 cos_2
# avg_sim_res_1(G1 vs G2) avg_sim_res_2(G2 vs G3) avg_lin_1 avg_lin_2 avg_rel_1 avg_rel_2 0     0
# max_sim_res_1(G1 vs G2) max_sim_res_2(G2 vs G3) max_lin_1 max_lin_2 max_rel_1 max_rel_2 0     0


print("Comparaciones de medidas por grupos, comparando contra G2")
print("   Measure           vs G1               vs G3       ")
print("------------- ------------------- -------------------")
print("resnik    min {sim1:.16f}  {sim2:.16f}".format(sim1=sims[0,0], sim2=sims[0,1]))
print("lin       min {sim1:.16f}  {sim2:.16f}".format(sim1=sims[0,2], sim2=sims[0,3]))
print("relevance min {sim1:.16f}  {sim2:.16f}".format(sim1=sims[0,4], sim2=sims[0,5]))
print("")
print("resnik    avg {sim1:.16f}  {sim2:.16f}".format(sim1=sims[1,0], sim2=sims[1,1]))
print("lin       avg {sim1:.16f}  {sim2:.16f}".format(sim1=sims[1,2], sim2=sims[1,3]))
print("relevance avg {sim1:.16f}  {sim2:.16f}".format(sim1=sims[1,4], sim2=sims[1,5]))
print("")
print("resnik    max {sim1:.16f}  {sim2:.16f}".format(sim1=sims[2,0], sim2=sims[2,1]))
print("lin       max {sim1:.16f}  {sim2:.16f}".format(sim1=sims[2,2], sim2=sims[2,3]))
print("relevance max {sim1:.16f}  {sim2:.16f}".format(sim1=sims[2,4], sim2=sims[2,5]))
print("")
print("cosine        {sim1:.16f}  {sim2:.16f}".format(sim1=sims[0,6], sim2=sims[0,7]))

Comparaciones de medidas por grupos, comparando contra G2
   Measure           vs G1               vs G3       
------------- ------------------- -------------------
resnik    min -0.0000000000000000  -0.0000000000000000
lin       min -0.0000000000000000  -0.0000000000000000
relevance min -0.0000000000000000  -0.0000000000000000

resnik    avg 0.0833016291382722  0.3741384035251289
lin       avg 0.2063489276867320  0.4184816018499463
relevance avg 0.0902849824594326  0.3048833827732753

resnik    max 0.2499048874148165  0.8068219201840938
lin       max 1.0000000000000000  1.0000000000000000
relevance max 0.4375355058616950  0.5949759444135203

cosine        0.0063271981687205  0.0111513939118874


## Análisis de resultados

*   En el caso de las medidas en los nodos que toman el valor mínimo, como los genes tienen una función en común esta medida da 0, porque se compara una función con si misma.
*   Las similitudes con promedio dan los valores esperados, mayor para la comparación con G3 en donde tienen un ancestro común más informativo.
*  Las medidas de resnik y relevance funcionan bien con el máximo, pero lin no, esto es porque la función está definida para devolver 1 si compara el mismo término.
*  La medida del coseno da mayor para la segunda comparación, que es lo que se esperaba.




# Tercer caso de prueba

En este caso los genes elegidos tienen más de una función en común:

![alt text](https://raw.githubusercontent.com/TiagoNLopez/stuff/master/go_test3.jpg)

In [0]:
G_go_id = [ ["GO:0051817","GO:0052249","GO:0052018","GO:0009616","GO:0060150"],
            ["GO:0051701","GO:0044003","GO:0052018","GO:0009616","GO:0060150"],
            ["GO:0051707","GO:0009615","GO:0098586","GO:0009616","GO:0060150"] ]

print("Gen 1:")
for func in G_go_id[0]:
   print(godag[func])
print(" ")

print("Gen 2:")
for func in G_go_id[1]:
   print(godag[func])
print(" ")

print("Gen 3:")
for func in G_go_id[2]:
   print(godag[func])

Gen 1:
GO:0051817	level-04	depth-04	modification of morphology or physiology of other organism involved in symbiotic interaction [biological_process]
GO:0052249	level-05	depth-05	modulation of RNA levels in other organism involved in symbiotic interaction [biological_process]
GO:0052018	level-06	depth-06	modulation by symbiont of RNA levels in host [biological_process]
GO:0009616	level-04	depth-07	virus induced gene silencing [biological_process]
GO:0060150	level-06	depth-09	viral triggering of virus induced gene silencing [biological_process]
 
Gen 2:
GO:0051701	level-04	depth-04	interaction with host [biological_process]
GO:0044003	level-05	depth-05	modification by symbiont of host morphology or physiology [biological_process]
GO:0052018	level-06	depth-06	modulation by symbiont of RNA levels in host [biological_process]
GO:0009616	level-04	depth-07	virus induced gene silencing [biological_process]
GO:0060150	level-06	depth-09	viral triggering of virus induced gene silencing [biologic

In [0]:
# min_sim_res_1(G1 vs G2) min_sim_res_2(G2 vs G3) min_lin_1 min_lin_2 min_rel_1 min_rel_2 cos_1 cos_2
# avg_sim_res_1(G1 vs G2) avg_sim_res_2(G2 vs G3) avg_lin_1 avg_lin_2 avg_rel_1 avg_rel_2 0     0
# max_sim_res_1(G1 vs G2) max_sim_res_2(G2 vs G3) max_lin_1 max_lin_2 max_rel_1 max_rel_2 0     0

sims = np.zeros((3,8))
res = []
lin = []
rel = []

for func1 in G_go_id[0]:
  for func2 in G_go_id[1]:
    res.append(resnik_sim(func1, func2, godag, termcounts))
    lin.append(lin_sim(func1, func2, godag, termcounts))
    rel.append(rel_sim(func1, func2, godag, termcounts))
    
sims[0,0]=min(res)
sims[1,0]=sum(res)/9
sims[2,0]=max(res)

sims[0,2]=min(lin)
sims[1,2]=sum(lin)/9
sims[2,2]=max(lin)

sims[0,4]=min(rel)
sims[1,4]=sum(rel)/9
sims[2,4]=max(rel)

res = []
lin = []
rel = []
for func1 in G_go_id[2]:
  for func2 in G_go_id[1]:
    res.append(resnik_sim(func1, func2, godag, termcounts))
    lin.append(lin_sim(func1, func2, godag, termcounts))
    rel.append(rel_sim(func1, func2, godag, termcounts))
    
sims[0,1]=min(res)
sims[1,1]=sum(res)/9
sims[2,1]=max(res)

sims[0,3]=min(lin)
sims[1,3]=sum(lin)/9
sims[2,3]=max(lin)

sims[0,5]=min(rel)
sims[1,5]=sum(rel)/9
sims[2,5]=max(rel)

ic_G1 = []
for func in G_go_id[0]:
  ic_G1.append(get_info_content(func,termcounts))

ic_G2 = []
for func in G_go_id[1]:
  ic_G2.append(get_info_content(func,termcounts))

ic_G3 = []
for func in G_go_id[2]:
  ic_G3.append(get_info_content(func,termcounts))
  
ic = [ ic_G1, ic_G2, ic_G3 ]

# sim coseno
  
G1v = [0,
       0, 0, 0, 0,
       0, 0, 0, 0, 0, 0,
       ic[0][0], 0, 0, 0, 0, 0,
       ic[0][1], 0, 0, 0, 0, 0, 0,
       ic[0][2], 0, 0, 0, 0,
       ic[0][3], 0,
       0,
       ic[0][4]]

G2v = [0,
       0, 0, 0, 0,
       0, 0, 0, 0, 0, 0,
       0, ic[1][0], 0, 0, 0, 0,
       0, ic[1][1], 0, 0, 0, 0, 0,
       ic[1][2], 0, 0, 0, 0,
       ic[1][3], 0,
       0,
       ic[1][4]]

G3v = [0,
       0, 0, 0, 0,
       0, 0, 0, 0, 0, 0,
       0, 0, ic[2][0], 0, 0, 0,
       0, 0, ic[2][1], 0, 0, 0, 0,
       0, 0, ic[2][2], 0, 0,
       ic[2][3], 0,
       0,
       ic[2][4]]

punto1 = np.array(G1v) @ np.array(G2v)
punto2 = np.array(G3v) @ np.array(G2v)

G1_snorm = 0
for x in G1v:
  G1_snorm += x**2
  
G2_snorm = 0
for x in G2v:
  G2_snorm += x**2
  
G3_snorm = 0
for x in G3v:
  G3_snorm += x**2
  
sims[0,6] = punto1 / (G1_snorm + G2_snorm - punto1)
sims[0,7] = punto2 / (G3_snorm + G2_snorm - punto2)

print("Comparaciones de medidas por grupos, comparando contra G2")
print("   Measure           vs G1               vs G3       ")
print("------------- ------------------- -------------------")
print("resnik    min {sim1:.16f}  {sim2:.16f}".format(sim1=sims[0,0], sim2=sims[0,1]))
print("lin       min {sim1:.16f}  {sim2:.16f}".format(sim1=sims[0,2], sim2=sims[0,3]))
print("relevance min {sim1:.16f}  {sim2:.16f}".format(sim1=sims[0,4], sim2=sims[0,5]))
print("")
print("resnik    avg {sim1:.16f}  {sim2:.16f}".format(sim1=sims[1,0], sim2=sims[1,1]))
print("lin       avg {sim1:.16f}  {sim2:.16f}".format(sim1=sims[1,2], sim2=sims[1,3]))
print("relevance avg {sim1:.16f}  {sim2:.16f}".format(sim1=sims[1,4], sim2=sims[1,5]))
print("")
print("resnik    max {sim1:.16f}  {sim2:.16f}".format(sim1=sims[2,0], sim2=sims[2,1]))
print("lin       max {sim1:.16f}  {sim2:.16f}".format(sim1=sims[2,2], sim2=sims[2,3]))
print("relevance max {sim1:.16f}  {sim2:.16f}".format(sim1=sims[2,4], sim2=sims[2,5]))
print("")
print("cosine        {sim1:.16f}  {sim2:.16f}".format(sim1=sims[0,6], sim2=sims[0,7]))

### Análisis de resultados:

*   En la medida de mínimo se toma la comparación entre los elementos iguales.
*   Las medidas de promedio dan resultados esperados.
*   En los casos de máximos, todas las medidas se quedan con el mayor resultado de la comparación de funciones iguales.
*   Coseno da el resultado esperado, mayor en el primer caso.

# Caso de prueba coseno

En este caso se busca comparar la medida de coseno con funciones de igual distribución pero a distintas profundidades del grafo.

![alt text](https://raw.githubusercontent.com/TiagoNLopez/stuff/master/go_test4.png)

In [0]:
G_go_id = [ ["GO:0050794","GO:0048522"],
            ["GO:0048518","GO:0048522"],
            ["GO:0016458","GO:0016441"],
            ["GO:0010608","GO:0016441"] ]

print("Gen 1:")
for func in G_go_id[0]:
   print(godag[func])
print(" ")

print("Gen 2:")
for func in G_go_id[1]:
   print(godag[func])
print(" ")

print("Gen 3:")
for func in G_go_id[2]:
   print(godag[func])
print(" ")
    
print("Gen 4:")
for func in G_go_id[3]:
   print(godag[func])

Gen 1:
GO:0050794	level-03	depth-03	regulation of cellular process [biological_process]
GO:0048522	level-04	depth-04	positive regulation of cellular process [biological_process]
 
Gen 2:
GO:0048518	level-03	depth-03	positive regulation of biological process [biological_process]
GO:0048522	level-04	depth-04	positive regulation of cellular process [biological_process]
 
Gen 3:
GO:0016458	level-02	depth-07	gene silencing [biological_process]
GO:0016441	level-03	depth-08	posttranscriptional gene silencing [biological_process]
 
Gen 4:
GO:0010608	level-06	depth-06	posttranscriptional regulation of gene expression [biological_process]
GO:0016441	level-03	depth-08	posttranscriptional gene silencing [biological_process]


In [0]:
ic_G1 = []
for func in G_go_id[0]:
  ic_G1.append(get_info_content(func,termcounts))

ic_G2 = []
for func in G_go_id[1]:
  ic_G2.append(get_info_content(func,termcounts))

ic_G3 = []
for func in G_go_id[2]:
  ic_G3.append(get_info_content(func,termcounts))

ic_G4 = []
for func in G_go_id[3]:
  ic_G4.append(get_info_content(func,termcounts))
  
ic = [ ic_G1, ic_G2, ic_G3, ic_G4 ]

# sim coseno
  
G1v = [0,
       0, 0,
       0, 0, 0,
       ic[0][0], 0, 0, 0, 0,
       ic[0][1], 0, 0, 0,
       0, 0,
       0, 0,
       0, 0,
       0, 0,
       0,
       0, 
       0]

G2v = [0,
       0, 0,
       0, 0, 0,
       0, ic[1][0], 0, 0, 0,
       ic[1][1], 0, 0, 0,
       0, 0,
       0, 0,
       0, 0,
       0, 0,
       0,
       0, 
       0]

G3v = [0,
       0, 0,
       0, 0, 0,
       0, 0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0,
       0, 0,
       ic[2][0], 0,
       0, ic[2][1],
       0,
       0, 
       0]

G4v = [0,
       0, 0,
       0, 0, 0,
       0, 0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0,
       0, 0,
       0, ic[3][0],
       0, ic[3][1],
       0,
       0, 
       0]

punto1 = np.array(G1v) @ np.array(G2v)
punto2 = np.array(G3v) @ np.array(G4v)

G1_snorm = 0
for x in G1v:
  G1_snorm += x**2
  
G2_snorm = 0
for x in G2v:
  G2_snorm += x**2
  
G3_snorm = 0
for x in G3v:
  G3_snorm += x**2
  
G4_snorm = 0
for x in G4v:
  G4_snorm += x**2
  
coseno_1 = punto1 / (G1_snorm + G2_snorm - punto1)
coseno_2 = punto2 / (G3_snorm + G4_snorm - punto2)

print("Coseno G1 vs G2: ", coseno_1)
print("Coseno G3 vs G4: ", coseno_2)

Coseno G1 vs G2:  0.48191553540485654
Coseno G3 vs G4:  0.4148858544240285
